In [3]:
import pandas as pd
import numpy as np
import os

In [248]:
path_homicidios = os.path.join('..','data','raw','homicidios.xlsx')

## ETL  Hechos

#### Extracción Hechos

In [249]:
hechos = pd.read_excel(path_homicidios,sheet_name='HECHOS')

In [250]:
hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


### Transformación Hechos


**Datos duplicados**

Observamos que en este caso, no tenemos nungun dato duplciado.

In [251]:
hechos[hechos['ID'].duplicated()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO


**Datos nulos**

A continuación analizaremos los datos nulos de la tabla hechos

In [252]:
pd.DataFrame(hechos.isna().sum(),columns=['Nulos'])

,Nulos
ID,0
N_VICTIMAS,0
FECHA,0
AAAA,0
MM,0
DD,0
HORA,0
HH,0
LUGAR_DEL_HECHO,0
TIPO_DE_CALLE,0


In [253]:
columns_with_nulls = ['Calle','Altura','Cruce','Dirección Normalizada']

- A continuación se observa que la columna altura y la columan Cruce contiene la mayor cantidad de nullos

- La columna Calle tiene un solo dato nulo.
- La columna Dirección Normalizada 8 nulos.
- La columna Cruce tiene $171$ nulos, siendo el 24,56% del los datos de la columna.
- La Altura tiene $567$ nulos que representan el 81,46% de datos de la columna


In [254]:
pd.DataFrame(hechos.isna().sum().loc[columns_with_nulls]/len(hechos),columns=['Porcentaje de nulos'])

,Porcentaje de nulos
Calle,0.001437
Altura,0.814655
Cruce,0.245690
Dirección Normalizada,0.011494


**Transformación de variables**
- Con la comunna altura y cruce podemos inferir que no pueden ocurrir ambas al mismo tiempo, por lo tanto si existe datos en una de ellas la otra deberá ser nula. Asi creamos una columna booleana, donde

$$\begin{cases}
1 && \text{Si el accidente fue en un cruce} \\
0 && \text{Si el accidente no fue en un cruce}\\
-1 && \text{No existe registro de la ubicación} 
\end{cases}$$

- Posteriormente vamos a remplazar los valores faltantes, con información de tipo de calle. por el momento dejaremos estos registros con -1

In [255]:
hechos['Cruce_boolean'] =  np.where(hechos['Cruce'].isnull() & hechos['Altura'].notnull(), 0,
        np.where(hechos['Cruce'].isnull() & hechos['Altura'].isnull(), -1,
        np.where(~hechos['Cruce'].isnull() & hechos['Altura'].isnull(), 1, np.nan)))

**Eliminación de columnas**

En esta sección eliminaremos las columnas que no utilizaremos en el proyecto.

- Dado que la dirección normalizada cuenta con la información del cruce o la altura respectivamente, procederemos a eliminarlas, ademas recordemos que contamos con la columna booleana que me informa si el accidente ocurrio en un cruce o no.

- La columna XY (CABA) nos podria servir para realizar una geolocalización de caba, pero se optó por buscar esta información en una base de datos oficial y posteriormente descargar un archivo geojson para realizar la vizualización del mapa de CABA, por lo tanto no necesitaremos esta columna para el analisis.  

In [256]:
hechos.drop(['Cruce','Altura', 'XY (CABA)'], axis=1,inplace=True)

**Tranformación de tipos** 

#### Llenado de nulos

- En el dataset se encuentran inconsistencias con la terminologia de los nullos, es por esto que se unificará con el string "SD" (sin datos). 

- En esta sección inspeccionaremos los datos para encontrar posibles nullos con este formato.

In [257]:
hechos['AAAA'].unique()
hechos['MM'].unique()
hechos['DD'].unique()
hechos['HORA'].unique()
hechos['HH'].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

- Anteriormente inspeccionamos si existia una fecha sin datos. Se observo que existe un dato sin hora (HORA) y sin franja horaria (HH). Este dato no es posible completarlo; dejaremos estos nulos ya que el objetivo es realizar un conteo de los usuarios que tuvieron un accidente fatal, es por esto que no eliminaremos registros. 

**Normalización de texto**

tomaremos las columnas con texto y las transformaremos en mayusculas.

In [258]:
hechos['LUGAR_DEL_HECHO'] = hechos['LUGAR_DEL_HECHO'].str.upper()
hechos['Calle'] = hechos['Calle'].str.upper()

Las calles que no tengan información las tomaremos como sin datos, ya que no tenemos información suficiente para inferirla

In [259]:
hechos.loc[hechos['Calle'].isna(),'Calle'] = 'SD'

La dirección normalizada es una concatenación de la información de la calle, avenida y cruce. por lo que a los datos nulos que tenemos en esta columna los llenaremos con la información del lugar del hecho.

In [260]:
hechos['Dirección Normalizada'] = hechos['Dirección Normalizada'].fillna(hechos['LUGAR_DEL_HECHO'])

In [261]:
hechos.isna().sum()

ID                       0
N_VICTIMAS               0
FECHA                    0
AAAA                     0
MM                       0
DD                       0
HORA                     0
HH                       0
LUGAR_DEL_HECHO          0
TIPO_DE_CALLE            0
Calle                    0
Dirección Normalizada    0
COMUNA                   0
pos x                    0
pos y                    0
PARTICIPANTES            0
VICTIMA                  0
ACUSADO                  0
Cruce_boolean            0
dtype: int64

In [262]:
hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Dirección Normalizada',
       'COMUNA', 'pos x', 'pos y', 'PARTICIPANTES', 'VICTIMA', 'ACUSADO',
       'Cruce_boolean'],
      dtype='object')

**Transformación de columna ubicación**

En esta sección vamos a limpiar las columnas con la latitud y longitud del accidente. se quiere extraer la ubicación de los datos validos, es por esto que un dato sin información coherente lo tomaremos como nulo.

In [263]:
valid_positions =  ~(hechos['pos x'].str.contains('[^0-9.,-]') | hechos['pos y'].str.contains('[^0-9.,-]') | (hechos['pos x'] == '.') | (hechos['pos y'] == '.'))

hechos.loc[~valid_positions, ['pos x', 'pos y']] = np.nan

In [264]:
hechos.loc[~valid_positions] 

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,Cruce_boolean
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",AUTOPISTA LUGONES PK 10000,13,NaN,NaN,MOTO-SD,MOTO,SD,-1.0
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,AU BUENOS AIRES - LA PLATA KM. 4,4,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,-1.0
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,SD,0,NaN,NaN,PEATON-SD,PEATON,SD,-1.0
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,0,NaN,NaN,SD-SD,SD,SD,-1.0
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV.",14,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,-1.0
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,9,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,-1.0
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,AU DELLEPIANE 2400,7,NaN,NaN,AUTO-AUTO,AUTO,AUTO,-1.0
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,NaN,NaN,MOTO-PASAJEROS,MOTO,PASAJEROS,-1.0
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",AUTOPISTA LUGONES KM 4.7,14,NaN,NaN,PEATON-AUTO,PEATON,AUTO,-1.0
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV. KM 6,1",14,NaN,NaN,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,-1.0


**Renombrar columnas**

In [265]:
nombres_columnas = {
    'ID': 'id_hechos',
    'N_VICTIMAS': 'n_victimas',
    'FECHA': 'fecha',
    'AAAA': 'anho',
    'MM': 'mes',
    'DD': 'dia',
    'HORA': 'hora',
    'HH': 'franja_horaria',
    'LUGAR_DEL_HECHO': 'lugar_del_hecho',
    'TIPO_DE_CALLE': 'tipo_de_calle',
    'Calle': 'calle',
    'Dirección Normalizada': 'direccion_normalizada',
    'COMUNA': 'comuna',
    'pos x': 'longitud',
    'pos y': 'latitud',
    'PARTICIPANTES': 'participantes',
    'VICTIMA': 'victimas',
    'ACUSADO': 'acusado',
}

hechos.rename(columns=nombres_columnas, inplace=True)

**Analisis finales de hechos**

In [266]:
hechos[hechos['Cruce_boolean']==-1].sample(10)

,id_hechos,n_victimas,fecha,anho,mes,dia,hora,franja_horaria,lugar_del_hecho,tipo_de_calle,calle,direccion_normalizada,comuna,longitud,latitud,participantes,victimas,acusado,Cruce_boolean
71,2016-0096,1,2016-07-25,2016,7,25,07:00:00,7,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,8,-58.47433193007387,-34.66684950051973,MOTO-CARGAS,MOTO,CARGAS,-1.0
398,2018-0124,1,2018-11-15,2018,11,15,19:40:00,19,AU PERITO MORENO Y RESERVISTAS ARGENTINOS,AUTOPISTA,AUTOPISTA PERITO MORENO,AUTOPISTA PERITO MORENO y RESERVISTAS ARGENTINOS,9,-58.52144411,-34.63682499,CARGAS-CARGAS,CARGAS,CARGAS,-1.0
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV. KM 6,1",14,NaN,NaN,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,-1.0
148,2017-0009,1,2017-01-16,2017,1,16,13:56:00,13,AU 25 DE MAYO Y AV. BOEDO,AUTOPISTA,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO y BOEDO AV.,5,-58.41595919,-34.62743346,MOTO-CARGAS,MOTO,CARGAS,-1.0
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,-58.37714647568196,-34.63657525428238,SD-SD,SD,SD,-1.0
281,2018-0007,1,2018-01-11,2018,1,11,02:30:00,2,AU PERITO MORENO KM 0.6,AUTOPISTA,AUTOPISTA PERITO MORENO,AUTOPISTA PERITO MORENO KM. 0.6,9,-58.52502765,-34.63601265,AUTO-AUTO,AUTO,AUTO,-1.0
585,2020-0066,1,2020-12-07,2020,12,7,04:30:00,4,AUTOPISTA PERITO MORENO ALTURA ESTACION DE TRE...,AUTOPISTA,AUTOPISTA PERITO MORENO,AUTOPISTA PERITO MORENO (ALTURA ESTACION DE TR...,10,-58.50178784,-34.63593897,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,-1.0
100,2016-0130,1,2016-10-04,2016,10,4,12:30:00,12,AUTOPISTA 9 DE JULIO SUR Y AV BRASIL,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI y B...,1,-58.37910942,-34.62764717,MOTO-CARGAS,MOTO,CARGAS,-1.0
601,2021-0003,1,2021-01-10,2021,1,10,07:40:00,7,"AU 25 DE MAYO KM. 3,7",AUTOPISTA,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO KM. 3.7,7,-58.43052224,-34.62941648,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,-1.0
513,2019-0097,1,2019-11-25,2019,11,25,08:50:00,8,"AUTOPISTA PERITO MORENO KM 5,6",AUTOPISTA,AUTOPISTA PERITO MORENO,AUTOPISTA PERITO MORENO KM. 5.6,9,-58.47721439,-34.64815069,MULTIPLE,MOTO,MULTIPLE,-1.0


- Con la información anteorio podemos inferir que si el tipo de calle es una autopista, no es posible que sea un cruce, por lo que remplazaremos estos datos con 0.

In [267]:
hechos.loc[(hechos['Cruce_boolean']==-1) & (hechos['tipo_de_calle'] == 'AUTOPISTA'),'Cruce_boolean'] = 0 

Observando el lugar_del_hecho, notamos que existen dos registros que tiene la información de la altura, por lo que procederemos a remplazar -1 por 0. a las victimas con ids 2017-0042 y 2020-0039

In [268]:
hechos[hechos['Cruce_boolean']==-1]

,id_hechos,n_victimas,fecha,anho,mes,dia,hora,franja_horaria,lugar_del_hecho,tipo_de_calle,calle,direccion_normalizada,comuna,longitud,latitud,participantes,victimas,acusado,Cruce_boolean
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,SD,0,NaN,NaN,PEATON-SD,PEATON,SD,-1.0
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV.",14,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,-1.0
559,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19,MURGUIONDO 2700,CALLE,MURGUIONDO,MURGUIONDO,9,NaN,NaN,PEATON-CARGAS,PEATON,CARGAS,-1.0


In [269]:
hechos.loc[(hechos['Cruce_boolean']==-1) & hechos['id_hechos'].isin(['2017-0042','2020-0039']), 'Cruce_boolean'] = 0 

In [270]:
hechos.loc[hechos['id_hechos'].isin(['2017-0042','2020-0039'])] 

,id_hechos,n_victimas,fecha,anho,mes,dia,hora,franja_horaria,lugar_del_hecho,tipo_de_calle,calle,direccion_normalizada,comuna,longitud,latitud,participantes,victimas,acusado,Cruce_boolean
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV.",14,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,0.0
559,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19,MURGUIONDO 2700,CALLE,MURGUIONDO,MURGUIONDO,9,NaN,NaN,PEATON-CARGAS,PEATON,CARGAS,0.0


In [271]:
hechos[hechos['Cruce_boolean']==-1]

,id_hechos,n_victimas,fecha,anho,mes,dia,hora,franja_horaria,lugar_del_hecho,tipo_de_calle,calle,direccion_normalizada,comuna,longitud,latitud,participantes,victimas,acusado,Cruce_boolean
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,SD,0,NaN,NaN,PEATON-SD,PEATON,SD,-1.0


Dado que solo tenemos un dato faltante y la victima es un peaton , remplazaremos con el dato más repetido en Cruce_boolean, ya que observamos que la mayoria de los homicidios de peatones ocurre en los cruces.

In [272]:
pd.DataFrame(hechos[hechos['victimas'] =='PEATON']['Cruce_boolean'].value_counts())

,count
Cruce_boolean,
1.0,207
0.0,56
-1.0,1


In [273]:
hechos.loc[hechos['id_hechos']=='2016-0151','Cruce_boolean'] = 1

**transformación de tipos**

In [274]:
hechos.dtypes

id_hechos                        object
n_victimas                        int64
fecha                    datetime64[ns]
anho                              int64
mes                               int64
dia                               int64
hora                             object
franja_horaria                   object
lugar_del_hecho                  object
tipo_de_calle                    object
calle                            object
direccion_normalizada            object
comuna                            int64
longitud                         object
latitud                          object
participantes                    object
victimas                         object
acusado                          object
Cruce_boolean                   float64
dtype: object

In [284]:
hechos['fecha'] = pd.to_datetime(hechos['fecha'], errors='coerce')  
hechos['hora'] = pd.to_datetime(hechos['hora'], format='%H:%M:%S', errors='coerce').dt.time  
hechos['longitud'] = pd.to_numeric(hechos['longitud'], errors='coerce')  
hechos['latitud'] = pd.to_numeric(hechos['latitud'], errors='coerce')
hechos['franja_horaria'] = pd.to_numeric(hechos['franja_horaria'], errors='coerce').astype('Int64') 
hechos['Cruce_boolean'] = hechos['Cruce_boolean'].astype('bool')  


In [285]:
hechos.dtypes

id_hechos                        object
n_victimas                        int64
fecha                    datetime64[ns]
anho                              int64
mes                               int64
dia                               int64
hora                             object
franja_horaria                    Int64
lugar_del_hecho                  object
tipo_de_calle                    object
calle                            object
direccion_normalizada            object
comuna                            int64
longitud                        float64
latitud                         float64
participantes                    object
victimas                         object
acusado                          object
Cruce_boolean                      bool
dtype: object

### Carga Hechos

In [287]:
path_hechos_clear = os.path.join('..','data','clear','hechos_homicidios.csv')

In [288]:
hechos.to_csv(path_hechos_clear,index=False)

# ETL Victimas

#### Extracción Victimas

In [4]:
path_homicidios = os.path.join('..','data','raw','homicidios.xlsx')

In [5]:
victimas = pd.read_excel(path_homicidios, sheet_name='VICTIMAS')

In [6]:
nombres_columnas_victimas = {
    'ID_hecho': 'id_hechos',
    'FECHA': 'fecha',
    'AAAA': 'anho',
    'MM': 'mes',
    'DD': 'dia',
    'ROL': 'rol',
    'VICTIMA': 'victimas',
    'SEXO': 'sexo',
    'EDAD': 'edad',
    'FECHA_FALLECIMIENTO': 'fecha_fallecimiento'
}

victimas.rename(columns=nombres_columnas_victimas, inplace=True)

In [7]:
victimas.head(5)

,id_hechos,fecha,anho,mes,dia,rol,victimas,sexo,edad,fecha_fallecimiento
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [8]:
victimas.columns

Index(['id_hechos', 'fecha', 'anho', 'mes', 'dia', 'rol', 'victimas', 'sexo',
       'edad', 'fecha_fallecimiento'],
      dtype='object')

#### Transformación Victimas

Transformaremos la columna fecha de falleciento para descartar la hora, ya que ningun registro cuenta la hora del homicidio.

In [9]:
victimas["fecha_fallecimiento"] = pd.to_datetime(victimas["fecha_fallecimiento"], format='%Y-%m-%d', errors='coerce')

In [10]:
victimas.dtypes

id_hechos                      object
fecha                  datetime64[ns]
anho                            int64
mes                             int64
dia                             int64
rol                            object
victimas                       object
sexo                           object
edad                           object
fecha_fallecimiento    datetime64[ns]
dtype: object

In [11]:
victimas["edad"] = victimas["edad"].replace(["SD"], np.nan)
victimas['edad'] = pd.to_numeric(victimas['edad'], errors='coerce')


In [12]:
victimas["edad"] = victimas["edad"].fillna(victimas["edad"].median())

In [13]:
victimas.head(10)

,id_hechos,fecha,anho,mes,dia,rol,victimas,sexo,edad,fecha_fallecimiento
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,NaT
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01
5,2016-0008,2016-01-24,2016,1,24,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-24
6,2016-0009,2016-01-24,2016,1,24,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,29.0,2016-01-26
7,2016-0010,2016-01-29,2016,1,29,CONDUCTOR,MOTO,MASCULINO,18.0,2016-01-29
8,2016-0012,2016-02-08,2016,2,8,CONDUCTOR,MOTO,MASCULINO,22.0,2016-02-08
9,2016-0013,2016-02-10,2016,2,10,PEATON,PEATON,MASCULINO,16.0,2016-02-10


**Creacion de columnas categoricas**

- Crearemos una columna para indicar si una persona murio el mismo dia del accidente o no. 

- Iniciaremos tratando los datos sin fecha. Tenemos 69 personas que no conocemos la fecha de fallecimiento la gran de datos del 2016. Dado que el objetivo principal es analisas la cantidad de accidentes fatales optamos por no eliminar registros, en este caso vamos a asumir que una persona que no tiene registro de fecha de fallecimiento falleció en el accidente, asumimos esto ya que por politicas de una entidad de salud si una persona ingresa a este lugar se debe tener registro de su fecha de fallecimiento, por lo que asumimos que si no tenemos registro de este suceso es por que la victima no pudo ingresar a la entidad prestadora de salud.

In [14]:
pd.DataFrame(victimas[victimas['fecha_fallecimiento'].isna()]['anho'].value_counts().reset_index().rename(columns={'count':'fallecidos sin fecha'}))

,anho,fallecidos sin fecha
0,2016,62
1,2017,6
2,2019,1


In [15]:

victimas.loc[victimas['fecha_fallecimiento'].isna(),'fecha_fallecimiento'] = victimas.loc[victimas['fecha_fallecimiento'].isna(),'fecha'] 

**Indicadora de fecha de fallecimiento**

Crearemos una columan que contenga 1 si la victima fallece el mismo dia del accidetente y 0 en caso contrario.

In [16]:
mask_fecha_fallecimiento = victimas['fecha_fallecimiento'] != victimas['fecha']

victimas[mask_fecha_fallecimiento].sample(10)

,id_hechos,fecha,anho,mes,dia,rol,victimas,sexo,edad,fecha_fallecimiento
398,2018-0108,2018-10-09,2018,10,9,CONDUCTOR,MOTO,MASCULINO,37.0,2018-10-10
73,2016-0097,2016-07-26,2016,7,26,PEATON,PEATON,MASCULINO,80.0,2016-08-12
442,2019-0008,2019-01-30,2019,1,30,CONDUCTOR,MOTO,MASCULINO,42.0,2019-01-31
716,2021-0096,2021-12-15,2021,12,15,CONDUCTOR,AUTO,MASCULINO,60.0,2021-12-20
648,2021-0029,2021-03-31,2021,3,31,CONDUCTOR,AUTO,MASCULINO,26.0,2021-04-02
346,2018-0058,2018-06-03,2018,6,3,CONDUCTOR,MOTO,MASCULINO,26.0,2018-06-13
409,2018-0119,2018-11-05,2018,11,5,PEATON,PEATON,FEMENINO,32.0,2018-11-06
684,2021-0065,2021-08-06,2021,8,6,PEATON,PEATON,FEMENINO,63.0,2021-08-10
41,2016-0054,2016-04-23,2016,4,23,CONDUCTOR,MOTO,MASCULINO,30.0,2016-04-25
430,2018-0139,2018-12-22,2018,12,22,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,28.0,2018-12-24


In [17]:
victimas['misma_fecha_accidente'] = ~ mask_fecha_fallecimiento

In [18]:
victimas.head(5)

,id_hechos,fecha,anho,mes,dia,rol,victimas,sexo,edad,fecha_fallecimiento,misma_fecha_accidente
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01,True
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02,True
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03,True
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,2016-01-10,True
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01,False


**Rango etario**

Crearemos con una columna para catalogar las personas como menor de edad, menor,joven, adulto, adulto mayor

In [19]:
bins = [0, 18, 30, 50, 100]

labels = ['menor', 'joven', 'adulto', 'adulto_mayor']

victimas['rango_etario'] = pd.cut(victimas['edad'], bins=bins, labels=labels, right=False) 


In [20]:
victimas.head(10)

,id_hechos,fecha,anho,mes,dia,rol,victimas,sexo,edad,fecha_fallecimiento,misma_fecha_accidente,rango_etario
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01,True,joven
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02,True,adulto_mayor
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03,True,adulto
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,2016-01-10,True,joven
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01,False,joven
5,2016-0008,2016-01-24,2016,1,24,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-24,True,adulto
6,2016-0009,2016-01-24,2016,1,24,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,29.0,2016-01-26,False,joven
7,2016-0010,2016-01-29,2016,1,29,CONDUCTOR,MOTO,MASCULINO,18.0,2016-01-29,True,joven
8,2016-0012,2016-02-08,2016,2,8,CONDUCTOR,MOTO,MASCULINO,22.0,2016-02-08,True,joven
9,2016-0013,2016-02-10,2016,2,10,PEATON,PEATON,MASCULINO,16.0,2016-02-10,True,menor


In [21]:
victimas.isna().sum()

id_hechos                0
fecha                    0
anho                     0
mes                      0
dia                      0
rol                      0
victimas                 0
sexo                     0
edad                     0
fecha_fallecimiento      0
misma_fecha_accidente    0
rango_etario             0
dtype: int64

In [28]:
def semestre(x):  
  if x<=6:
    return 1
  else:
    return 2 
  

victimas['semestre'] = victimas['mes'].apply(semestre) 

In [31]:
victimas.head(4)

,id_hechos,fecha,anho,mes,dia,rol,victimas,sexo,edad,fecha_fallecimiento,misma_fecha_accidente,rango_etario,semestre
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01,True,joven,1
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02,True,adulto_mayor,1
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03,True,adulto,1
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,2016-01-10,True,joven,1


In [30]:
path_victimas_clear = os.path.join('..','data','clear','victimas_homicidios.csv')

victimas.to_csv(path_victimas_clear, index=False)


# ETL Lesiones

##### Extracción Lesiones

In [23]:
path_lesiones = os.path.join('..','data','raw','lesiones.xlsx')

In [24]:
lesiones = pd.read_excel(path_lesiones)

In [25]:
lesiones.sample(10)

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
6479,LC-2019-0485943,1,2019,8,6,2019-08-06 00:00:00,11:00:00,11,SD,No Especificada,...,SD,SD,SD,SD-SD,x,SD,SD,SD,x,SD
21901,LC-2021-0501238,1,2021,10,8,2021-10-08 00:00:00,22:10:00,22,RIVADAVIA AV. y AZUL,10,...,-34.632315660528,SD,MOTO,SD-MOTO,x,SD,SD,SD,SD,GRAVE
2184,LC-2019-0181214,1,2019,3,23,2019-03-23 00:00:00,21:16:00,21,SD,1,...,-34.613433,PEATON,SD,PEATON-SD,SD,SD,x,SD,SD,SD
11864,LC-2020-0111139,1,2020,2,20,2020-02-20 00:00:00,09:02:00 a.m.,9,ESMERALDA 1000,1,...,-34.596403633558,PEATON,CAMION,PEATON-CAMION,0,0,0,x,0,GRAVE
4201,LC-2019-0320626,1,2019,5,22,2019-05-22 00:00:00,17:30:00,17,SD,1,...,-34.619762,SD,SD,SD-SD,x,SD,SD,SD,SD,SD
4574,LC-2019-0347089,1,2019,6,4,2019-06-04 00:00:00,00:16:00,0,SD,14,...,-34.587581,SD,SD,SD-SD,x,SD,SD,SD,SD,SD
13836,LC-2020-0496422,1,2020,8,27,2020-08-27 00:00:00,02:30:00 p.m.,14,"LINIERS VIRREY y CALVO, CARLOS",5,...,-34.623333159239,PEATON,SD,PEATON-SD,0,0,0,0,0,SD
19895,LC-2021-0320645,1,2021,7,7,2021-07-07 00:00:00,23:15:00,23,"PAZ, GRAL. AV. y ALBERDI, JUAN BAUTISTA AV.",9,...,-34.662524884133,PEATON,MOTO,PEATON-MOTO,x,0,0,0,0,SD
21283,LC-2021-0467688,1,2021,9,9,2021-09-09 00:00:00,18:00:00,18,SAN JUAN AV. y COMBATE DE LOS POZOS,3,...,-34.622821800135,TRANSPORTE PUBLICO,TRANSPORTE PUBLICO,TRANSPORTE PUBLICO-TRANSPORTE PUBLICO,0,0,x,0,0,SD
23692,LC-2021-0644046,1,2021,12,27,2021-12-27 00:00:00,10:50:00,10,"ENSENADA y ALBERDI, JUAN BAUTISTA AV.",10,...,-34.636488264505,AUTO,TRANSPORTE PUBLICO,AUTO-TRANSPORTE PUBLICO,0,x,x,0,0,SD


#### Transformación Lesiones

#### Exportar Lesiones